In [56]:
import torch
import torch.optim
import pandas as pd
import os

import nnts
import nnts.data
from nnts import utils
import nnts.torch.models
import nnts.torch.trainers
import nnts.metrics
import nnts.torch.datasets
import nnts.torch.utils
import nnts.loggers
from nnts import datasets
torch.set_printoptions(precision=8, sci_mode=False)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
data_path = "data"
model_name = ""
dataset_name = "ETTh1"
results_path = "nb-results"
metadata_path = "informer.json"

metadata = datasets.load_metadata(dataset_name, path=metadata_path)
datafile_path = os.path.join(data_path, metadata.filename)
PATH = os.path.join(results_path, model_name, metadata.dataset)
df = pd.read_csv(datafile_path)
utils.makedirs_if_not_exists(PATH)

In [39]:
# params = utils.Hyperparams(
#     optimizer=torch.optim.Adam,
#     loss_fn=torch.nn.L1Loss(),
#     batch_size=32,
#     batches_per_epoch=50,
#     training_method=utils.TrainingMethod.DMS,
#     model_file_path="logs",
#     epochs=100,
#     scheduler=utils.Scheduler.REDUCE_LR_ON_PLATEAU,
# )

In [58]:
df = df.rename({"date": "ds", "OT": "y"}, axis="columns")
df['unique_id'] = 'T1'
#df = df[['unique_id', 'ds', 'y']]

In [41]:
#ONLY RUN FOR MULTIVARIATE
#df = pd.melt(df, id_vars=["date"], value_vars=["HUFL", "HULL", "MUFL", "MULL", "LUFL", "LULL", "OT"], var_name="unique_id", value_name="y")
#df = df.rename({"date": "ds"}, axis="columns")
#df.head()

In [59]:
# split lengths as per informer
# trn_length = int(24 * 365.25)
# val_test_length = int(24 * 365.25 * (4 / 12))
nnts.torch.utils.seed_everything(2021)
trn_length = 8640
val_test_length = 2880
split_data = datasets.split_test_val_train(
    df, trn_length, val_test_length, val_test_length
)

In [60]:

import nnts.torch.datasets
import nnts.torch.models.dlinear
import nnts.torch.utils

params = nnts.torch.models.tsmixer.get_mutlivariate_params()
params.enc_in = 1
params.individual = False

dataset_options = {
    "context_length": metadata.context_length,
    "prediction_length": metadata.prediction_length,
    "conts": [
         "HUFL",
         "HULL",
         "MUFL",
         "MULL",
         "LUFL",
         "LULL",
    ],
}

trn_dl, val_dl, test_dl = nnts.torch.utils.create_dataloaders_from_split_data(
    split_data,
    #Dataset=nnts.torch.datasets.TimeseriesDataset,
    Dataset=nnts.torch.datasets.MultivariateTimeSeriesDataset,
    dataset_options=dataset_options,
    #Sampler=nnts.torch.datasets.TimeSeriesSampler,
    batch_size=params.batch_size,
    transforms = [nnts.torch.preprocessing.StandardScaler()]
)

net = nnts.torch.models.TSMixer(metadata.context_length, metadata.prediction_length,7, params)


#net = nnts.torch.models.DLinear(
#    metadata.prediction_length,
#    metadata.context_length,
#    7,
#    params
#)

In [61]:
trner = nnts.torch.trainers.ValidationTorchEpochTrainer(net, params, metadata)
evaluator = trner.train(trn_dl, val_dl)
y_hat, y = evaluator.evaluate(
    test_dl, metadata.prediction_length, metadata.context_length
)


TSMixer(
  (mixer_layers): Sequential(
    (0): MixerLayer(
      (time_mixing): TimeMixing(
        (norm): TimeBatchNorm2d(2352, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dropout): Dropout(p=0.9, inplace=False)
        (fc1): Linear(in_features=336, out_features=336, bias=True)
      )
      (feature_mixing): FeatureMixing(
        (norm_before): TimeBatchNorm2d(2352, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (norm_after): Identity()
        (dropout): Dropout(p=0.9, inplace=False)
        (fc1): Linear(in_features=7, out_features=256, bias=True)
        (fc2): Linear(in_features=256, out_features=7, bias=True)
        (projection): Identity()
      )
    )
    (1): MixerLayer(
      (time_mixing): TimeMixing(
        (norm): TimeBatchNorm2d(2352, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dropout): Dropout(p=0.9, inplace=False)
        (fc1): Linear(in_features=336, out_features=336, bias=True

In [62]:
y_hat.shape, y.shape

(torch.Size([2545, 336, 7]), torch.Size([2545, 336, 7]))

In [63]:
nnts.metrics.calculate_seasonal_error(trn_dl, metadata.seasonality).shape

torch.Size([1, 1])

In [64]:
def MAE(pred, true):
    return torch.mean(torch.abs(pred - true))


def MSE(pred, true):
    return torch.mean((pred - true) ** 2)

In [65]:
MAE(y_hat, y), MSE(y_hat, y)

(tensor(0.65725952), tensor(0.77339315))

In [66]:

test_metrics = nnts.metrics.calc_metrics(
    y_hat, y, nnts.metrics.calculate_seasonal_error(trn_dl, metadata.seasonality)
)
test_metrics

{'mse': 0.7733930349349976,
 'abs_error': 3934250.25,
 'abs_target_sum': 4788612.5,
 'abs_target_mean': 0.7999901175498962,
 'seasonal_error': 0.4050425887107849,
 'mean_mase': 1.622692346572876,
 'mean_mape': 5.055292129516602,
 'mean_smape': 1.336384654045105,
 'mean_msmape': 1.0481796264648438,
 'mean_mae': 0.6572595238685608,
 'mean_rmse': 0.8372031450271606,
 'median_mase': 1.5489574670791626,
 'median_smape': 1.4468520879745483,
 'median_msmape': 1.0248525142669678,
 'median_mae': 0.6273937225341797,
 'median_rmse': 0.7912198901176453}

In [21]:
0.3049468696117401**2

0.09299259328599963

In [ ]:
nnts.metrics.calc_metrics(y_hat[:, :1, :], y[:, :1, :], nnts.metrics.calculate_seasonal_error(trn_dl, metadata))

In [ ]:
def save_results(y_hat, y, name):
    torch.save(y_hat, f"{PATH}/{name}_y_hat.pt")
    torch.save(y, f"{PATH}/{name}_y.pt")
save_results(y_hat, y, scenario.name)

In [ ]:
covariate_name = f"cov-1-pearsn-0.68-pl-{str(scenario.prediction_length)}-seed-{scenario.seed}"
covariate_y_hat = torch.load(f"{PATH}/{covariate_name}_y_hat.pt")
covariate_y = torch.load(f"{PATH}/{covariate_name}_y.pt")

In [ ]:
def calculate_forecast_horizon_metrics(y_hat, y, metadata, metric="mae"):
    forecast_horizon_metrics = []
    for i in range(1, metadata.prediction_length):
        metrics = nnts.metrics.calc_metrics(y[:, :i, :], y_hat[:, :i, :], metadata.freq, metadata.seasonality)
        forecast_horizon_metrics.append(metrics[metric])
    return forecast_horizon_metrics

forecast_horizon_metrics = calculate_forecast_horizon_metrics(y_hat, y, metadata, "mae")
covariate_forecast_horizon_metrics = calculate_forecast_horizon_metrics(covariate_y_hat, covariate_y, metadata, "mae")

In [ ]:
import seaborn as sns
sns.set()

In [ ]:
plt.plot(forecast_horizon_metrics, label='univariate')
plt.plot(covariate_forecast_horizon_metrics, label='covariate (0.68)')
plt.xlabel("Forecast Horizon")
plt.ylabel("Error (MAE)")
plt.legend()
plt.show()

In [ ]:
csv_aggregator = nnts.datasets.CSVFileAggregator(PATH, "results")
results = csv_aggregator()

In [ ]:
df.tail(metadata.prediction_length*50)['y'].plot()

In [ ]:
y_hat_last = y_hat[:, :1, ...]
y_last = y[:, :1, ...]
df_test = df.tail(y_hat_last.shape[0])
df_test["y_check"] = y_last.squeeze()
df_test["y_hat"] = y_hat_last.squeeze()
df_test[["y", "y_check", "y_hat"]]
df_test.set_index("ds")[["y_check", "y_hat"]].iloc[4500:4500+336].plot(figsize=(20, 10))

In [ ]:
test_metrics = nnts.metrics.calc_metrics(y_last, y_hat_last, metadata.freq, metadata.seasonality)
test_metrics